# In this notebook we will explore some essentials functions to work with text in spacy v3.


In [118]:
import pandas as pd

#Import the requisite library
import spacy
print(spacy.__version__)
nlp = spacy.load("es_core_news_sm")

import re

3.3.0


## Features de Texto con spacy

In [16]:
#Sample text
text = "Recibí por mail un aviso que alguien ingresó con mi usuario desde Nueva York, mi numero de documento es 11223344. Además me dijeron que no podia hacer nada, y que me aumentan el precio. Voy a llamar a defensa del consumidor"
doc= nlp(text)

In [17]:
for sent in doc.sents:
    print(sent.text) 

Recibí por mail un aviso que alguien ingresó con mi usuario desde Nueva York, mi numero de documento es 11223344.
Además me dijeron que no podia hacer nada, y que me aumentan el precio.
Voy a llamar a defensa del consumidor


In [18]:
[sent.text for sent in doc.sents]

['Recibí por mail un aviso que alguien ingresó con mi usuario desde Nueva York, mi numero de documento es 11223344.',
 'Además me dijeron que no podia hacer nada, y que me aumentan el precio.',
 'Voy a llamar a defensa del consumidor']

In [19]:
for sent in doc.sents:
    for token in sent:
        print(token.text, token.lemma_, token.pos_)

Recibí Recibí PROPN
por por ADP
mail mail VERB
un uno DET
aviso aviso NOUN
que que PRON
alguien alguien PRON
ingresó ingresar VERB
con con ADP
mi mi DET
usuario usuario NOUN
desde desde ADP
Nueva Nueva PROPN
York York PROPN
, , PUNCT
mi mi DET
numero numero NOUN
de de ADP
documento documento NOUN
es ser AUX
11223344 11223344 NOUN
. . PUNCT
Además además ADV
me yo PRON
dijeron decir VERB
que que SCONJ
no no ADV
podia podiar VERB
hacer hacer VERB
nada nada PRON
, , PUNCT
y y CCONJ
que que SCONJ
me yo PRON
aumentan aumentar VERB
el el DET
precio precio NOUN
. . PUNCT
Voy voy AUX
a a ADP
llamar llamar VERB
a a ADP
defensa defensa NOUN
del del ADP
consumidor consumidor NOUN


## matcher con spacy

In [20]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

In [21]:
sustantivos = [[{'POS': 'NOUN' or 'PROPN'}]]
matcher.add("SUSTANTIVOS", sustantivos)

In [22]:

sustantivos = matcher(doc)
for match_id, start, end in sustantivos:
    print("Match Found: ", doc[start:end].text)

Match Found:  aviso
Match Found:  usuario
Match Found:  numero
Match Found:  documento
Match Found:  11223344
Match Found:  precio
Match Found:  defensa
Match Found:  consumidor


In [23]:

matcher2 = Matcher(nlp.vocab)

patterns = [
    [{"TEXT": {"REGEX": "((\d){8})"}}], #numeros de documento
    [{"LOWER": "no"}, {"TEXT": {"REGEX": "\w+"}, "OP": "?"}, {"LEMMA": "poder"}, {"TEXT": {"REGEX": "\w+"}, "OP": "?"}, {"TEXT": {"REGEX": "^comunicar"}}], 
    [{"LEMMA": {"IN": ['aumentar','modificar']}}, {"TEXT": {"REGEX": "\w+"}, "OP": "?"}, {"TEXT": {"REGEX": "\w+"}, "OP": "?"}, {"TEXT": {"REGEX": "\w+"}, "OP": "?"}, {"lower": {"IN": ['precio','abono']}}], #  Modificar / Aumentar el precio / abono
    [{"LEMMA": {"REGEX": "defensa|aten[sc]i[oó]n"}}, {"TEXT": {"REGEX": "\w+"}, "OP": "?"}, {"TEXT": {"REGEX": "\w+"}, "OP": "?"}, {"TEXT": {"REGEX": "\w+"}, "OP": "?"}, {"lower": {"REGEX": "consumid"}}] #  Defensa del consumidor
    ] #cosas mas especificas



matcher2.add("TEXT_PATTERNS", patterns)

patrones = matcher2(doc)
for match_id, start, end in patrones:
    print("Match Found: ", doc[start:end].lemma_)



Match Found:  11223344
Match Found:  aumentar el precio
Match Found:  defensa del consumidor


## Regex dentro del texto con spacy

In [25]:
import re

In [116]:
doc = nlp("Estamos robando un monton, pero no es mucho, vamos robando mucho mucho mucho mas")


expression = r"\brobando\W+(?:\w+\W+){0,3}?mucho\b" #SIMULA NEAR
expression = r"\brobando\ mucho\b" #UNA PALABRA AL LADO DE LA OTRA


for match in re.finditer(expression, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end)
    # This is a Span object or None if match doesn't map to valid token sequence
    if span is not None:
        print("Found match:", span.text)

Found match: robando mucho
